[View in Colaboratory](https://colab.research.google.com/github/NicknameSoHard/Open_course_ML/blob/master/ML_homework_6.ipynb)

In [1]:
!wget https://raw.githubusercontent.com/Yorko/mlcourse_open/master/data/winequality-white.csv -P drive/Open_course_ML  

--2018-07-31 14:23:21--  https://raw.githubusercontent.com/Yorko/mlcourse_open/master/data/winequality-white.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 264426 (258K) [text/plain]
Saving to: ‘drive/Open_course_ML/winequality-white.csv’

winequality-white.c 100%[===================>] 258.23K  --.-KB/s    in 0.03s   

2018-07-31 14:23:21 (8.72 MB/s) - ‘drive/Open_course_ML/winequality-white.csv’ saved [264426/264426]



In [0]:
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**Будем работать с набором данных по качеству белого вина (репозиторий UCI). Загружаем данные.**

In [0]:
data = pd.read_csv('drive/Open_course_ML/winequality-white.csv', sep=';')

In [180]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.**

In [0]:
x_columns = data.columns.values
x_columns =  [x for x in data if x != 'quality']

X = data[ x_columns ]
y = data['quality']

X_train, X_holdout, y_train, y_holdout = train_test_split( X, y, test_size=0.3, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Линейная регрессия

**Обучите простую линейную регрессию.**

In [200]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

**<font color='red'>Вопрос 1:</font> Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?**

In [201]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, linreg.predict(X_train_scaled) ) )
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled) ) )

Mean squared error (train): 0.558
Mean squared error (test): 0.584


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). **<br>
**<font color='red'>Вопрос 2:</font> Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?**

In [0]:
def print_top_coeff(value_coefficient, position):
  attribute = dict()
  i = 0
  for name in X.columns.values:
    attribute.update( {name : np.abs(value_coefficient[i]) })
    i+=1

  sorted_dict_keys = sorted( attribute, key = attribute.get )[::-1]
  for key in sorted_dict_keys:
    print(key, attribute[key])

  print('\nОтвет:', sorted_dict_keys[position] )

In [185]:
print_top_coeff(linreg.coef_ , 0)

density 0.6657204718353074
residual sugar 0.5381640961630233
volatile acidity 0.19225994745411332
pH 0.1500360058866863
alcohol 0.1295334473979827
fixed acidity 0.09782192229174835
sulphates 0.06205336047703555
free sulfur dioxide 0.04218044058626405
total sulfur dioxide 0.014304022736369677
chlorides 0.008127243529684174
citric acid 0.00018322444870894072

Ответ: density


## Lasso-регресссия

**Обучите Lasso-регрессию с небольшим коэффициентом α=0.01 (слабая регуляризация). Пусть опять random_state=17.**

In [186]:
lasso1 = Lasso( alpha=0.01,  random_state=17 )
lasso1.fit(X_train_scaled, y_train) 

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=17,
   selection='cyclic', tol=0.0001, warm_start=False)

**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**

In [187]:
print_top_coeff( lasso1.coef_ , -1)

alcohol 0.32242504702545427
residual sugar 0.2563628135082211
density 0.2354918517265469
volatile acidity 0.18847921541595028
pH 0.06727654842362442
free sulfur dioxide 0.043087618524916715
sulphates 0.029721520382053737
chlorides 0.002747319653577024
total sulfur dioxide 0.0
citric acid 0.0
fixed acidity 0.0

Ответ: fixed acidity


**Теперь определите лучшее значение α в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.**

In [188]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV( alphas=alphas, cv=5, random_state=17)
lasso_cv.fit(X_train_scaled, y_train) 

LassoCV(alphas=array([1.00000e-06, 1.09699e-06, ..., 9.11589e+01, 1.00000e+02]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
    verbose=False)

In [189]:
lasso_cv.alpha_

0.0002833096101839324

**Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина. **<br>
**<font color='red'>Вопрос 3:</font> Какой признак "обнулился первым" в настроенной модели LASSO?**

In [190]:
lasso1 = Lasso( alpha=lasso_cv.alpha_,  random_state=17 )
lasso1.fit(X_train_scaled, y_train) 

Lasso(alpha=0.0002833096101839324, copy_X=True, fit_intercept=True,
   max_iter=1000, normalize=False, positive=False, precompute=False,
   random_state=17, selection='cyclic', tol=0.0001, warm_start=False)

In [191]:
print_top_coeff(lasso1.coef_ , -1)

density 0.6481609673722674
residual sugar 0.5268830668067311
volatile acidity 0.19204855840369864
pH 0.14654888754244963
alcohol 0.137114781992208
fixed acidity 0.09329523893868766
sulphates 0.06093920359373391
free sulfur dioxide 0.04269847076590106
total sulfur dioxide 0.01296897671264232
chlorides 0.006932920797374191
citric acid 0.0

Ответ: citric acid


**Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках..**<br>
**<font color='red'>Вопрос 4:</font> Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?**

In [192]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, lasso1.predict(X_train_scaled) ))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, lasso1.predict(X_holdout_scaled) ))

Mean squared error (train): 0.558
Mean squared error (test): 0.583


## Случайный лес

**Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.**

In [193]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=17, verbose=0, warm_start=False)

**<font color='red'>Вопрос 5:</font>  Каковы среднеквадратичные ошибки случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?**

In [194]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, forest.predict(X_train_scaled)) )
print("Mean squared error (cv): %.3f" % np.abs(np.mean(cross_val_score(forest, X_train_scaled, 
                                                              y_train, scoring='neg_mean_squared_error'))))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, forest.predict(X_holdout_scaled) ))


Mean squared error (train): 0.075
Mean squared error (cv): 0.460
Mean squared error (test): 0.421


**Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.**

In [195]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(forest, forest_params, n_jobs=-1 )
locally_best_forest.fit(X_train_scaled, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7], 'max_features': [6, 7, 8, 9, 10, 11]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [196]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 22, 'max_features': 7, 'min_samples_leaf': 1},
 0.42345378900629316)

**К сожалению, результаты GridSearchCV не полностью воспроизводимы (могут отличаться на разных платформах даже при фиксировании random_state). Поэтому обучите лес с параметрами max_depth=19, max_features=7, и min_samples_leaf=1 (лучшие в моем случае).**<br>
**<font color='red'>Вопрос 6:</font> Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?**

In [197]:
forest = RandomForestRegressor(max_depth=19, max_features=7, min_samples_leaf=1, random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=19,
           max_features=7, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=17,
           verbose=0, warm_start=False)

In [198]:
print("Mean squared error (cv): %.3f" % np.abs(np.mean(cross_val_score(forest, X_train_scaled, 
                                                              y_train, scoring='neg_mean_squared_error'))))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, forest.predict(X_holdout_scaled) ))

Mean squared error (cv): 0.457
Mean squared error (test): 0.410


**Оцените важность признаков с помощью случайного леса.**<br>
**<font color='red'>Вопрос 7:</font> Какой признак оказался главным в настроенной модели случайного леса?**

In [199]:
print_top_coeff( forest.feature_importances_ , 0)

alcohol 0.224431837517167
volatile acidity 0.11939289880404694
free sulfur dioxide 0.11614718944344686
pH 0.07280565100192624
total sulfur dioxide 0.07131778876323688
residual sugar 0.07015969761778447
density 0.06936694368363032
chlorides 0.06798170918739124
fixed acidity 0.0642679551222412
citric acid 0.0629447776288302
sulphates 0.061183551230298616

Ответ: alcohol


**Сделайте выводы о качестве моделей и оценках влияния признаков на качество вина с помощью этих трех моделей.**

В нашей текущей задаче Линейная и LASSO регресии получили практически идентичные результаты, единственное только вторая потребовала дополнительного подбора обощающего коэффициента, за счет чего стала менее предпочтительной для этой задани.
Лучше всего справился случайный лес с решением, несмотря на то что ему так же пришлось проводить кросс-валидацию. Но зато мы получили решение, которое ошибается на 20 процентов меньше.

![alt text](https://pp.userapi.com/c831309/v831309216/151c5e/bKxOx3kI32I.jpg)